In [31]:
#!sudo apt-get install libgomp1
!pip install faiss-cpu --no-cache
#!pip install transformers faiss torch
!pip install sentence-transformers

     |████████████████████████████████| 7.9 MB 2.2 MB/s eta 0:00:01


In [34]:
from transformers import AutoModel, AutoTokenizer
import pandas as pd
import torch
import faiss
import numpy as np

In [3]:
# Using the community model
# BERT Base
tokenizer = AutoTokenizer.from_pretrained('neuralmind/bert-base-portuguese-cased')
model = AutoModel.from_pretrained('neuralmind/bert-base-portuguese-cased')

In [19]:
df = pd.read_csv('./Analise metricas/Datasets a serem usados/dataset_tratado.csv')

In [22]:
documents = df['perguntas'].tolist()[:1000]
len(documents)

1000

In [24]:
vectors = [
  # tokenize the document, return it as PyTorch tensors (vectors),
  # and pass it onto the model
  model(**tokenizer(document, return_tensors='pt'))[0].detach().squeeze()
  for document in documents
]

[v.size() for v in vectors]

[torch.Size([12, 768]),
 torch.Size([10, 768]),
 torch.Size([10, 768]),
 torch.Size([11, 768]),
 torch.Size([10, 768]),
 torch.Size([12, 768]),
 torch.Size([9, 768]),
 torch.Size([12, 768]),
 torch.Size([9, 768]),
 torch.Size([9, 768]),
 torch.Size([13, 768]),
 torch.Size([10, 768]),
 torch.Size([9, 768]),
 torch.Size([10, 768]),
 torch.Size([14, 768]),
 torch.Size([12, 768]),
 torch.Size([10, 768]),
 torch.Size([14, 768]),
 torch.Size([14, 768]),
 torch.Size([23, 768]),
 torch.Size([13, 768]),
 torch.Size([18, 768]),
 torch.Size([22, 768]),
 torch.Size([13, 768]),
 torch.Size([16, 768]),
 torch.Size([15, 768]),
 torch.Size([18, 768]),
 torch.Size([7, 768]),
 torch.Size([17, 768]),
 torch.Size([13, 768]),
 torch.Size([27, 768]),
 torch.Size([25, 768]),
 torch.Size([27, 768]),
 torch.Size([13, 768]),
 torch.Size([11, 768]),
 torch.Size([15, 768]),
 torch.Size([18, 768]),
 torch.Size([35, 768]),
 torch.Size([12, 768]),
 torch.Size([15, 768]),
 torch.Size([14, 768]),
 torch.Size([19, 768]

In [28]:
averaged_vectors = [torch.mean(vector, dim=0) for vector in vectors]

[v.size() for v in averaged_vectors]

[torch.Size([768]),
 torch.Size([768]),
 torch.Size([768]),
 torch.Size([768]),
 torch.Size([768]),
 torch.Size([768]),
 torch.Size([768]),
 torch.Size([768]),
 torch.Size([768]),
 torch.Size([768]),
 torch.Size([768]),
 torch.Size([768]),
 torch.Size([768]),
 torch.Size([768]),
 torch.Size([768]),
 torch.Size([768]),
 torch.Size([768]),
 torch.Size([768]),
 torch.Size([768]),
 torch.Size([768]),
 torch.Size([768]),
 torch.Size([768]),
 torch.Size([768]),
 torch.Size([768]),
 torch.Size([768]),
 torch.Size([768]),
 torch.Size([768]),
 torch.Size([768]),
 torch.Size([768]),
 torch.Size([768]),
 torch.Size([768]),
 torch.Size([768]),
 torch.Size([768]),
 torch.Size([768]),
 torch.Size([768]),
 torch.Size([768]),
 torch.Size([768]),
 torch.Size([768]),
 torch.Size([768]),
 torch.Size([768]),
 torch.Size([768]),
 torch.Size([768]),
 torch.Size([768]),
 torch.Size([768]),
 torch.Size([768]),
 torch.Size([768]),
 torch.Size([768]),
 torch.Size([768]),
 torch.Size([768]),
 torch.Size([768]),


In [29]:
def encode(document: str) -> torch.Tensor:
    tokens = tokenizer(document, return_tensors='pt')
    vector = model(**tokens)[0].detach().squeeze()
    return torch.mean(vector, dim=0)

In [35]:
index = faiss.IndexIDMap(faiss.IndexFlatIP(768)) # the size of our vector space
# index all the documents, we need them as numpy arrays first
index.add_with_ids(
    np.array([t.numpy() for t in averaged_vectors]),
    # the IDs will be 0 to len(documents)
    np.array(range(0, len(documents)))
)


In [36]:
def search(query: str, k=1):
    encoded_query = encode(query).unsqueeze(dim=0).numpy()
    top_k = index.search(encoded_query, k)
    scores = top_k[0][0]
    results = [documents[_id] for _id in top_k[1][0]]
    return list(zip(results, scores))

In [37]:
documents[0]

'o covid-19 terminara em breve?'

In [38]:
search(documents[0],k=2)

[('o covid-19 terminara em breve?', 49.250393),
 ('o covid-19 terminara no verao?', 47.493435)]

In [43]:
search('Quanto tempo vai durar essa pandemia?',k=5)

[('quanto tempo voce acha que a pandemia covid-19 vai durar na america?',
  36.097225),
 ('quanto tempo leva para as pessoas infectadas apresentarem os sintomas?',
  34.69768),
 ('quanto tempo vai demorar para os cientistas desenvolverem uma vacina contra covid-19?',
  34.353043),
 ('o que e uma pandemia?', 33.73018),
 ('o covid-19 vai durar ate que todos morram e os humanos sejam extintos?',
  33.702965)]